In [1]:
machine_name = "valve"

# Import Libaries

In [2]:
import numpy as np
import pandas as pd
import pickle
import os
import shutil
import matplotlib.pyplot as plt

## Create directory management

In [3]:
# If dir not present creatre
pickel_dir = "saved_model"
if not os.path.exists(pickel_dir):
    os.makedirs(pickel_dir)

In [4]:
from sklearn.metrics import classification_report

def class_report(y_test, y_pred):
    target_names = []
    for i in y.unique():
        target_names.append(f"class {i}")
        
    try:
        y_true = y_test[0].tolist()
        
    except:
        y_true = y_test.tolist()
        
    y_pred = y_pred.tolist()
    print(classification_report(y_true, y_pred, target_names=target_names))

## Funnel feature

In [5]:
important_col = ['is_normal', 'T_rms_mean', 'T_rms_std', 'T_zcr_mean', 'T_zcr_std',
                   'F_mel_mean', 'F_mel_std', 'F_mel_rms_mean', 'F_mel_rms_std',
                   'F_mfcc_mean', 'F_mfcc_std', 'F_flatness_mean', 'F_flatness_std',
                   'F_bandwidth_mean', 'F_bandwidth_std', 'F_contrast_mean',
                   'F_contrast_std', 'F_rolloff_mean', 'F_rolloff_std']

dataset = pd.read_csv(f'./dataset/processed_data/{machine_name}_all.csv.xz')
dataset = dataset[important_col]

In [6]:
len(dataset.columns)

19

## Split data

In [7]:
X = dataset.drop("is_normal", axis=1)
y = dataset["is_normal"]

from sklearn.model_selection import train_test_split

X_model, X_val, y_model, y_val  = train_test_split(X, y, test_size=0.1, random_state=1)

## Resample over sample

In [8]:
from imblearn.over_sampling import BorderlineSMOTE
X_resample, y_resample = BorderlineSMOTE().fit_resample(X_model, y_model.ravel())

y_resample = pd.DataFrame(y_resample)
X_resample = pd.DataFrame(X_resample)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resample, y_resample, test_size=0.2, random_state=1)

## Random Forest Estimator 100

In [9]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [10]:
from sklearn.ensemble import RandomForestClassifier

no_estimators = 100

RF_classifier = RandomForestClassifier(random_state = 0, n_estimators = no_estimators, criterion = 'entropy')
RF_classifier.fit(X_train, y_train.values.ravel())

RandomForestClassifier(criterion='entropy', random_state=0)

In [11]:
print("************************  Test ************************")
y_pred = RF_classifier.predict(X_test)
class_report(y_test, y_pred)

print("************************  Validation ************************")
y_val_pred = RF_classifier.predict(X_val)
class_report(y_val, y_val_pred)

************************  Test ************************
              precision    recall  f1-score   support

     class 0       0.95      0.96      0.95      1958
     class 1       0.96      0.95      0.96      2030

    accuracy                           0.96      3988
   macro avg       0.96      0.96      0.96      3988
weighted avg       0.96      0.96      0.96      3988

************************  Validation ************************
              precision    recall  f1-score   support

     class 0       0.66      0.78      0.71       148
     class 1       0.97      0.95      0.96      1103

    accuracy                           0.93      1251
   macro avg       0.82      0.86      0.84      1251
weighted avg       0.93      0.93      0.93      1251



## Saving Machine model to predict abnormal or normal sound

In [12]:
filename = 'Predict_slider_sound_type.sav'
pickle.dump(RF_classifier, open(f"./{pickel_dir}/{filename}", 'wb'))

# load the model from disk
loaded_model = pickle.load(open(f"./{pickel_dir}/{filename}", 'rb'))
result = loaded_model.score(X_val, y_val)
print(round(result,2))

0.93


# 

## K-mean clustering
* Sub catogorize abnormal dataset.

## Using the elbow method to find the optimal number of clusters

In [13]:
important_col = ['is_normal', 'T_rms_mean', 'T_rms_std', 'T_zcr_mean',
                   'F_mel_mean', 'F_mel_std', 'F_mel_rms_mean', 'F_mel_rms_std',
                   'F_mfcc_mean', 'F_mfcc_std', 'F_flatness_mean',
                   'F_bandwidth_mean', 'F_bandwidth_std', 'F_contrast_mean',
                   'F_rolloff_mean', 'F_rolloff_std']

cluster_dataset = pd.read_csv(f'./dataset/processed_data/{machine_name}_all.csv.xz')
cluster_dataset = cluster_dataset[important_col]
abnormal_dataset = cluster_dataset[cluster_dataset.is_normal == 0]
dataset = abnormal_dataset.drop(columns=["is_normal"])

In [ ]:
from sklearn.cluster import KMeans
wcss = []
total_test = 20
random_S = 0
for i in range(1, total_test):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = random_S)
    kmeans.fit(dataset)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, total_test), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS');

## Training the K-Means model on the dataset

In [ ]:
no_cluster = 4
kmeans = KMeans(n_clusters = no_cluster, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(dataset)

## Saving Machine model to predict abnormal sound type

In [ ]:
filename = f'Predict_{machine_name}_abnormal_sound_type.sav'
pickle.dump(kmeans, open(f"./{pickel_dir}/{filename}", 'wb'))

# load the model from disk
loaded_model = pickle.load(open(f"./{pickel_dir}/{filename}", 'rb'))